In [105]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

import csv
import time

scrapedata = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

scrapedata.get("https://www.scrapethissite.com/pages/forms/?page_num=1")

#let's get the table headers
thead = scrapedata.find_elements(By.TAG_NAME, "th")
table_head = []
for th in thead:
    table_head.append(th.text)

file = open('Advanced_Scraping_Selenium.csv', 'w',  newline='', encoding='UTF8')
writer = csv.writer(file)
writer.writerow(table_head)

# let's get the rows 
#row_data = scrapedata.find_elements(By.CLASS_NAME, "team")

while True:
    row_data = scrapedata.find_elements(By.CLASS_NAME, "team")
    for team in row_data:
        cel_row = team.find_elements(By.TAG_NAME, "td")
        table_row = [ cel.text for cel in cel_row]
        writer.writerow(table_row)
    try: #to get the next page
        scrapedata.find_element(By.PARTIAL_LINK_TEXT, "»").click()
        time.sleep(2)
    except NoSuchElementException:
        break
        

scrapedata.quit()
file.close()

In [107]:
#Check out the data
import pandas as pd

df= pd.read_csv(r'Advanced_Scraping_Selenium.csv')

In [111]:
df.head()

,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,NaN,0.550,299,264,35
1,Buffalo Sabres,1990,31,30,NaN,0.388,292,278,14
2,Calgary Flames,1990,46,26,NaN,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,NaN,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,NaN,0.425,273,298,-25


In [143]:
#let's get the team with most wins

total_wins =df.groupby(['Team Name'])[['Wins','Losses']].sum()

total_wins_sorted = total_wins.sort_values(by='Wins', ascending=False)

total_wins_sorted['Total Games'] = total_wins_sorted['Wins'] + total_wins_sorted['Losses']
total_wins_sorted['NewWin %'] = total_wins_sorted['Wins'] / total_wins_sorted['Total Games']
total_wins_sorted.head()

,Wins,Losses,Total Games,NewWin %
Team Name,,,,
Detroit Red Wings,986,480,1466,0.672578
New Jersey Devils,905,564,1469,0.616065
Pittsburgh Penguins,839,642,1481,0.566509
Philadelphia Flyers,836,598,1434,0.582985
Boston Bruins,816,614,1430,0.570629


In [159]:
#let's get the most win per year

max_year =df.groupby(['Year'])['Wins'].max()
most_wins = pd.merge(df, max_year, on = ['Year', 'Wins'], how = 'inner')

most_wins.head()

,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Chicago Blackhawks,1990,49,23,NaN,0.613,284,211,73
1,New York Rangers,1991,50,25,NaN,0.625,321,246,75
2,Pittsburgh Penguins,1992,56,21,NaN,0.667,367,268,99
3,New York Rangers,1993,52,24,NaN,0.619,299,231,68
4,Detroit Red Wings,1994,33,11,NaN,0.688,180,117,63


In [163]:
repeated_teams = most_wins['Team Name'].value_counts()
repeated_teams

Team Name
Detroit Red Wings      6
New York Rangers       3
Pittsburgh Penguins    2
Colorado Avalanche     2
Dallas Stars           2
Vancouver Canucks      2
Chicago Blackhawks     1
St. Louis Blues        1
Ottawa Senators        1
Buffalo Sabres         1
Boston Bruins          1
San Jose Sharks        1
Washington Capitals    1
Name: count, dtype: int64